In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization

In [ ]:
# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:/Users/MSI KATANA/Desktop/practice ML/Computer-vision-projects-by-Inupa-Bandara/Nike vs adidas shoe classification/train',
    image_size=(224, 128),  # Resize to 128x128 to match the example
    batch_size=32,
    shuffle=False  # Keep the data in order to easily split by class
)